In [1]:
import pandas as pd
import numpy as np
import gudhi as gd

In [3]:
heatmap_1_1 = pd.read_csv('heatmap/heatmap_1.csv', index_col=0)
heatmap_1_1000 = pd.read_csv('heatmap/heatmap_1_1000.csv', index_col=0)
heatmap_1_2000 = pd.read_csv('heatmap/heatmap_1_2000.csv', index_col=0)

In [5]:
heatmap_2_1 = pd.read_csv('heatmap/heatmap_2.csv', index_col=0)
heatmap_2_1000 = pd.read_csv('heatmap/heatmap_2_1000.csv', index_col=0)
heatmap_2_2000 = pd.read_csv('heatmap/heatmap_2_2000.csv', index_col=0)

In [6]:
combined_1 = heatmap_1_1.combine_first(heatmap_1_1000)
combined_1 = combined_1.combine_first(heatmap_1_2000)

In [8]:
combined_2 = heatmap_2_1.combine_first(heatmap_2_1000)
combined_2 = combined_2.combine_first(heatmap_2_2000)

In [10]:
combined_1.to_csv('heatmap_1.csv', index=True)
combined_2.to_csv('heatmap_2.csv', index=True)

In [11]:
def find_similar(heatmap, refcode, threshold):
    similar = {}
    refcode_col = heatmap.loc[refcode]
    # normalising the scale of values and ranging similarity from 1 (very similar) to 0 (least similar) and take out the value corresponding to refcode
    norm_values = refcode_col.sort_values().transform(lambda x:(refcode_col.max()-x)/refcode_col.max())
    indices = norm_values.index.values.tolist()
    # after being sorted, the second element has the highest similarity value (doesn't mean necessarily they are similar in absolute terms)
    best_max = norm_values.loc[indices[1]]
    similar[indices[0]] = heatmap.loc[refcode, indices[0]]
    # take out the element corresponding to best_max
    norm_values = norm_values.iloc[1:]
    for index in indices[1:]:
        if  norm_values.loc[index] >= threshold*best_max:
            similar[index]=heatmap.loc[refcode][index]

    return similar

In [12]:
similar_1_AGAPAA = find_similar(combined_1, 'AGAPAA', 0.95)
similar_1_AHABOA = find_similar(combined_1, 'AHABOA', 0.95)
similar_1_BOYJOP = find_similar(combined_1, 'BOYJOP', 0.95)

similar_2_AGAPAA = find_similar(combined_2, 'AGAPAA', 0.95)
similar_2_AHABOA = find_similar(combined_2, 'AHABOA', 0.95)
similar_2_BOYJOP = find_similar(combined_2, 'BOYJOP', 0.95)

In [17]:
similar = (
    set(similar_1_AGAPAA.keys()) & set(similar_2_AGAPAA.keys()) | 
    set(similar_1_AHABOA.keys()) & set(similar_2_AHABOA.keys()) | 
    set(similar_1_BOYJOP.keys()) & set(similar_2_BOYJOP.keys())
)

In [20]:
no_betti = pd.read_csv('heatmap/no_betti_2.csv')

In [23]:
no_betti_crystal = list(no_betti['GAMQOB']) + ['GAMQOB']

In [24]:
crystal_removal = list(similar) + no_betti_crystal

In [27]:
extracted_crystal = list(set(combined_1.index) - set(crystal_removal))

In [30]:
with open('potential_cages_list.txt', 'w') as file:
    for crystal in extracted_crystal:
        file.write(crystal + '\n')